In [ ]:
import pdfplumber
import pandas as pd
import numpy as np
import shutil
import os
import datetime
import locale
import requests
from tqdm import tqdm
locale.setlocale(locale.LC_ALL, 'es_ES.utf8')

In [ ]:
def download_file(url):
    local_filename = url.split('/')[-1]

    with requests.get(url) as r:
        with open(local_filename, 'wb') as f:
            f.write(r.content)
        
    return local_filename

In [ ]:
path = os.getcwd()
folder = "ddjj_pdf"
if os.path.exists(os.path.join(path, folder)) != True:
        os.mkdir(folder)

folder_path = os.path.join(path, folder)

In [ ]:
data=pd.read_excel("ddjj_url.xlsx")

In [ ]:
'''
ANDERSON HINT
Caso de que no exista el archivo no devolver la accion
'''


def return_data_pdf_status(file:str)->any:
    try:
        temp = pdfplumber.open(file)
        return temp
    except Exception as e:
        print(f"{'-'*20}\t{file}\t{'-'*20}")
        print(e)
        
        return None


files=os.listdir()
files_pdf=[]
for file in files:
    if file.find(".pdf") != -1:
        files_pdf.append(file)
files_pdf

In [ ]:
dic={}
format = "%A %d de %B de %Y"

for url in tqdm(data.url.to_numpy()): #files_pdf
    file = download_file(url)

    #TODO -> validacion que no es necesaria usar si no se encuentra el archivo 
    temp = return_data_pdf_status(file)
    if temp is None:
        continue

    totalpages = len(temp.pages)
    lista=[]
    for i in range(0,totalpages):
        page = temp.pages[i]
        if i == 0:
            try: 
                fecha = page.annots[0]["data"]["V"].decode("latin1")
                dt = datetime.datetime.strptime(fecha,format)
                year = str(dt.year)
                dt = dt.strftime("%d/%m/%Y")
            except:
                fecha = page.annots[1]["data"]["V"].decode("latin1")
                dt = datetime.datetime.strptime(fecha,format)
                year = str(dt.year)
                dt = dt.strftime("%d/%m/%Y")
        prueba=pd.Series(page.extract_text().split("\n"))
        if i != totalpages - 1:
            prueba=prueba.str.split(":",expand=True)
        lista.append(prueba)
    for j in range(0,len(lista)):
        if j == 0:
            total = lista[j]
        else:
            total=pd.concat([total,lista[j]],axis=0)
    total=total.reset_index(drop=True)
    
    data = total.drop(total.index[range(0,7)]).reset_index(drop=True)
    lista=[]
    c=0
    for i,var in enumerate(data[1]):
        if var == None:
            valor=data[0].iloc[i]
        lista.append(valor) 
    data["informacion"]=lista
    for i,var in enumerate(data[0]):
        if var.find("Declaro") != -1 and c == 0:
            c = 1
            index = i
            data["informacion"][index:len(data[0])]=np.nan
    for value1,value2,idx in zip(data[0],data.informacion,data.index):
        if value1 == value2:
            data[1][idx]="-"
        else:
            continue
    mask = data[1] == "-"
    data = data[np.logical_not(mask)]
    data["periodo"] = year
    data["presentacion"] = dt
    data= data.rename(columns={0:"tipo_de_dato",
                              1:"valor"})
    data = data[["informacion","tipo_de_dato","valor","presentacion","periodo"]]
    dic[file.replace(".pdf","")]=data
    temp.close()
    shutil.move(file,folder)

In [ ]:
data=pd.DataFrame()
for key in tqdm(dic.keys()):
    data = pd.concat([data,dic[key]],axis=0)
data.to_excel("ddjj.xlsx",index=False)
data.to_csv("ddjj.csv",index=False)
shutil.rmtree(folder)